# Predict Tomorrows Bitcoin Close Price using Classical Machine Learning

In [7]:

df, predictions = run_prediction_pipeline(quant_path = r'C:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\data\20251110_234455\quant\quant_bitcoin_test_20251110_2344.csv',
    google_path= r'C:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\data\20251110_234455\sentiment\google_news_sentiment_20251110_2357_days_20.csv',
    interest_path= r'C:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\data\20251110_234455\interest\interest_rates_test_20251110_2357.csv',
    models_dir = MODELS_DIRECTORY)
prompt = get_prompt(predictions, df['close'].iloc[-1])

--- Starting Data Preprocessing with Temporal Alignment ---
--- Data Preprocessing Complete ---
--- Starting Prediction ---
--- Starting Prediction Pipeline ---
Loading artifacts from: C:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\models\20251109_210801
--- Performing Feature Engineering ---

Forecast for the Next Day:
  - Linear Regression: 105479.08
  - Ridge Regression: 105517.47
  - Random Forest: 101043.87
  - XGBoost: 98323.48


In [ ]:
from helpers.data_ingestation import get_interest_data, get_quant_data,extract_google_sentiment
from main import run_prediction_pipeline
from helpers.llm_support import get_prompt

print("Forecasting Tomorrows Bitcoin Close Price...")
MODELS_DIRECTORY = r'C:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\models\20251109_210801'
df, predictions = run_prediction_pipeline(MODELS_DIRECTORY)
prompt = get_prompt(predictions, df['close'].iloc[-1])

c:\Users\baile\anaconda3\envs\crypto_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Forecasting Tomorrows Bitcoin Close Price...

Step 1: Ingesting fresh data...
Saving this run's data under: c:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\data\20251110_234455
--- Fetching Quant Data ---


Fetching OHLCV:  95%|█████████▌| 1641600000/1728000000 [00:01<00:00, 925482612.47ms/s]
Device set to use cpu


Quant data saved to: c:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\data\20251110_234455\quant\quant_bitcoin_test_20251110_2344.csv


Batch sentiment: 100%|██████████| 44/44 [04:51<00:00,  6.63s/it]


✅ Google sentiment data saved to: c:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\data\20251110_234455\sentiment\google_news_sentiment_20251110_2357_days_20.csv
--- Fetching Interest Rate Data (Year-by-Year) ---
  -> Fetching: 2024-11-10 to 2025-11-10

🎉 Total Unique Records Fetched: 204
Interest rate data saved to: c:\Users\baile\Documents\Artificial Intelligence\BitcoinPred\data\20251110_234455\interest\interest_rates_test_20251110_2357.csv
--- Starting Data Preprocessing with Temporal Alignment ---


ValueError: Invalid file path or buffer object type: <class 'tuple'>

## Feed the Classical Machine Learning Predictions into the LLM to have it search the web with context. 

In [5]:
predictions

{'Linear Regression': 105479.08486911432,
 'Ridge Regression': 105517.47075229813,
 'Random Forest': 101043.87093333338,
 'XGBoost': 98323.48}

In [ ]:
from google import genai
from google.genai import types
from helpers.queries import urls
from helpers.llm_support import add_citations
from dotenv import load_dotenv #comment this out
import os

# Load .env file
load_dotenv()
api_key = os.getenv("gemini_api_key") #replace with your api key
client = genai.Client()

# Build the tools list with each URL as url_context
tools = [
  {"url_context": {}}, #specific user defined urls
  types.Tool(google_search=types.GoogleSearch()) #dynamic search
]

urls = urls() #pulls from dependency

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[prompt + "Look through these and some of your own URLs not listed here: {urls}"],
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=-1), #dynamically decides how much it needs to think
        temperature=0.0,
        tools=tools
    )
)

text_with_citations = add_citations(response)
print(text_with_citations)

ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}